In [1]:
%reload_ext autoreload
%autoreload 2

import os

os.chdir(f"/home/{os.getlogin()}/watttime-python-client-aer-algo")

import math
import numpy as np
import pandas as pd
import datetime
from pytz import UTC, timezone
import seaborn as sns
from datetime import datetime, timedelta
import concurrent.futures

from watttime import WattTimeForecast, WattTimeHistorical, RecalculatingWattTimeOptimizer

import data.s3 as s3u
import evaluation.eval_framework as efu
from plotnine import *

username = os.getenv("WATTTIME_USER")
password = os.getenv("WATTTIME_PASSWORD")

actual_data = WattTimeHistorical(username, password)
hist_data = WattTimeForecast(username, password)

s3 = s3u.s3_utils()

## Initialize parameters

In [2]:
tz = "America/Chicago"

In [3]:
region = "SPP_TX"
start_time = datetime(2023, 7, 11, 20, 6)
start_time = UTC.localize(start_time)
end_time = start_time + timedelta(hours = 12)
usage_time = 120
usage_power = 2
optimization_method="simple"
requery_interval_minutes = 60

## Collect historical MOER data every hour to pass to recalculating optimizer

In [4]:
forecast_generator = WattTimeForecast(username, password)

In [5]:
moer_data_list = []

new_start_time = start_time

while new_start_time < end_time:
            
    new_data = forecast_generator.get_historical_forecast_pandas(
                start=new_start_time - timedelta(minutes=5), 
                end=new_start_time,
                region=region,
                signal_type="co2_moer",
                horizon_hours=72,
            )
    new_data["point_time"] = pd.to_datetime(new_data["point_time"])
    moer_data_list.append(new_data)
            
    new_start_time = new_start_time + timedelta(minutes = requery_interval_minutes)

## Run recalculating optimizer by passing MOER data

In [6]:
opt_test_moer_list = RecalculatingWattTimeOptimizer(region = region, 
                               watttime_username = username, 
                               watttime_password = password,
                               usage_time_required_minutes=usage_time,
                               usage_power_kw=usage_power,
                               optimization_method=optimization_method)

for curr_fcst_data in moer_data_list:
    new_start_time = pd.to_datetime(curr_fcst_data["point_time"]).min()
    print(new_start_time)
    opt_test_moer_list.get_new_schedule(new_start_time=new_start_time, new_end_time=end_time, curr_fcst_data=curr_fcst_data)

2023-07-11 20:05:00+00:00
== Simple fit! ==
2023-07-11 21:05:00+00:00
== Simple fit! ==
2023-07-11 22:05:00+00:00
== Simple fit! ==
2023-07-11 23:05:00+00:00
== Simple fit! ==
2023-07-12 00:05:00+00:00
== Simple fit! ==
2023-07-12 01:05:00+00:00
== Simple fit! ==
2023-07-12 02:05:00+00:00
== Simple fit! ==
2023-07-12 03:05:00+00:00
== Simple fit! ==
2023-07-12 04:05:00+00:00
== Simple fit! ==
2023-07-12 05:05:00+00:00
== Simple fit! ==
2023-07-12 06:05:00+00:00
== Simple fit! ==
2023-07-12 07:05:00+00:00
== Simple fit! ==


## Run recalculating optimizer by hitting api

In [7]:
opt_test_api = RecalculatingWattTimeOptimizer(region = region, 
                               watttime_username = username, 
                               watttime_password = password,
                               usage_time_required_minutes=usage_time,
                               usage_power_kw=usage_power,
                               optimization_method=optimization_method)

new_start_time = start_time

while new_start_time < end_time:
    print(new_start_time)
    opt_test_api.get_new_schedule(new_start_time=new_start_time, new_end_time=end_time)
    new_start_time = new_start_time + timedelta(minutes = requery_interval_minutes)

2023-07-11 20:06:00+00:00


== Simple fit! ==
2023-07-11 21:06:00+00:00
== Simple fit! ==
2023-07-11 22:06:00+00:00
== Simple fit! ==
2023-07-11 23:06:00+00:00
== Simple fit! ==
2023-07-12 00:06:00+00:00
== Simple fit! ==
2023-07-12 01:06:00+00:00
== Simple fit! ==
2023-07-12 02:06:00+00:00
== Simple fit! ==
2023-07-12 03:06:00+00:00
== Simple fit! ==
2023-07-12 04:06:00+00:00
== Simple fit! ==
2023-07-12 05:06:00+00:00
== Simple fit! ==
2023-07-12 06:06:00+00:00
== Simple fit! ==
2023-07-12 07:06:00+00:00
== Simple fit! ==


## Difference in outputs - note the row count

In [11]:
opt_test_moer_list.get_combined_schedule()

,pred_moer,usage,emissions_co2e_lb,energy_usage_mwh
point_time,,,,
2023-07-11 20:05:00+00:00,1366.2,0.0,0.000000,0.000000
2023-07-11 20:10:00+00:00,1366.2,0.0,0.000000,0.000000
2023-07-11 20:15:00+00:00,1365.7,0.0,0.000000,0.000000
2023-07-11 20:20:00+00:00,1365.2,0.0,0.000000,0.000000
2023-07-11 20:25:00+00:00,1364.5,0.0,0.000000,0.000000
...,...,...,...,...
2023-07-12 07:45:00+00:00,141.8,5.0,0.023633,0.000167
2023-07-12 07:50:00+00:00,147.3,0.0,0.000000,0.000000
2023-07-12 07:55:00+00:00,154.0,0.0,0.000000,0.000000


In [12]:
opt_test_api.get_combined_schedule()

,pred_moer,usage,emissions_co2e_lb,energy_usage_mwh
point_time,,,,
2023-07-11 20:10:00+00:00,1366.2,0.0,0.000000,0.000000
2023-07-11 20:15:00+00:00,1365.7,0.0,0.000000,0.000000
2023-07-11 20:20:00+00:00,1365.2,0.0,0.000000,0.000000
2023-07-11 20:25:00+00:00,1364.5,0.0,0.000000,0.000000
2023-07-11 20:30:00+00:00,1364.8,0.0,0.000000,0.000000
...,...,...,...,...
2023-07-12 07:45:00+00:00,141.8,5.0,0.023633,0.000167
2023-07-12 07:50:00+00:00,147.3,5.0,0.024550,0.000167
2023-07-12 07:55:00+00:00,154.0,5.0,0.025667,0.000167


## Missing point times are hourly at the edge between periods

In [13]:
opt_test_moer_list.get_combined_schedule()[~opt_test_moer_list.get_combined_schedule().index.isin(opt_test_api.get_combined_schedule().index)]

,pred_moer,usage,emissions_co2e_lb,energy_usage_mwh
point_time,,,,
2023-07-11 20:05:00+00:00,1366.2,0.0,0.000000,0.000000
2023-07-11 21:05:00+00:00,1352.8,0.0,0.000000,0.000000
2023-07-11 22:05:00+00:00,1350.0,0.0,0.000000,0.000000
2023-07-11 23:05:00+00:00,1337.3,0.0,0.000000,0.000000
2023-07-12 00:05:00+00:00,1350.2,0.0,0.000000,0.000000
2023-07-12 01:05:00+00:00,1347.8,0.0,0.000000,0.000000
2023-07-12 02:05:00+00:00,1352.1,0.0,0.000000,0.000000
2023-07-12 03:05:00+00:00,1390.7,0.0,0.000000,0.000000
2023-07-12 04:05:00+00:00,1361.4,0.0,0.000000,0.000000
